Tutorials table of content:

- [Tutorial 1: Run a first scenario](./Tutorial-1_Run_your_first_scenario.ipynb)
- [Tutorial 2: Add contributivity measurements methods](./Tutorial-2_Add_contributivity_measurement.ipynb)
- Tutorial 3: Use a custom dataset


# Tutorial 3 : Use homemade dataset 

With this example, we dive deeper into the potential of the library, and run a scenario on a new dataset, that we will implement 

## 1 - Prerequisites

In order to run this example, you'll need to:

* use python 3.7 +
* install this package https://pypi.org/project/mplc/

If you did not follow our firsts tutorials, it is highly recommended to [take a look at it !](https://github.com/SubstraFoundation/distributed-learning-contributivity/tree/master/notebooks/examples/) 


In [ ]:
!pip install mplc

## 2 - Context 

In collaborative data science projects partners sometimes need to train a model on multiple datasets, contributed by different data providing partners. In such cases the partners might have to measure how much each dataset involved contributed to the performance of the model. This is useful for example as a basis to agree on how to share the reward of the ML challenge or the future revenues derived from the predictive model, or to detect possible corrupted datasets or partners not playing by the rules. The library explores this question and the opportunity to implement some mechanisms helping partners in such scenarios to measure each dataset's *contributivity* (as *contribution to the performance of the model*).

In the [first tutorial](./Tutorial-1_Run_your_first_scenario.ipynb), you learned how to parametrize and run a scenario.
In the [second tutorial](./Tutorial-2_Add_contributivity_measurement.ipynb), you discovered how to add to your scenario run one of the contributivity measurement methods available.
In this third tutorial, we are going to use a custom dataset.  

### The dataset : Sentiment140
We are going to use a subset of the [sentiment140](http://help.sentiment140.com/for-students) dataset and try to 
classified short film review, between positive sentiments and negative sentiments for movies. 

*The whole machine learning process is inspired from this [article](https://medium.com/@alyafey22/sentiment-classification-from-keras-to-the-browser-7eda0d87cdc6)*
Please note that the library provided a really easy way to adapt a single partner, common machine learning use case with tensorflow, to a multipartner case, with contributivity measurement. 

In [2]:
# imports
import seaborn as sns
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

import re

from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding

from mplc.dataset import Dataset
from mplc.scenario import Scenario

sns.set()

Using TensorFlow backend.


170500096/170498071 [==============================] - 11s 0us/step


## 3 - Generation, and preparation of the dataset
 
The scenario object needs a dataset object to run. In the previous tutorials, we indicate which one to generate automatically by passing a name of a pre-implemented dataset to the scenario constructor. 
Here, we will create this dataset object and pass it to the scenario constructor. To do so, we are going to create a new class, which inherit from the mplc.Dataset abstract class.

A sub-class of Dataset needs few attribute and method. First, the constructor of the Dataset object needs few arguments.
### Dataset generator :

The structure of the dataset generator is represented below:

```python
dataset = Dataset(
    "name",
    x_train,
    x_test,
    y_train,
    y_test,
    input_shape,
    num_classes,
)
```
#### Data labels
The data labels can take whatever shape you need, with only one condition. 
The labels need to be convertible into string format, and with respect to the condition that if label1 is equal to label2 (
reciprocally different from), therefore str(label1) must be equal to str(label2) (reciprocally different from)
#### Model generator
This method needs to be implemented, and provides the model use, which will be trained by the `Scenario` object.
Note: It is mandatory to have loss and accuracy as metrics for your model.

#### Train/validation/test splits

The `Dataset` constructor (called via `super()`)  must be provided some separated train and test sets (referred to as global train set and global test set).
The global train set is then further split into a global train set and a global validation set, by the function `train_val_split_global`. Please denote that if this function is not overwritten, the sklearn's `train_test_split` function will be called by default, and 10% of the training set will be use as validation set.
In the multi-partner learning computations, the global validation set is used for early stopping and the global test set is used for performance evaluation.
The global train set is then split amongst partners (according to the scenario configuration) to populate the partner's local datasets.
For each partner, the local dataset will be split into separated train, validation and test sets, using the `train_test_split_local` and `train_val_split_local` methods.
These are not mandatory, by default the local dataset will not be split. 
Denote that currently, the local validation and test set are not used, but they are available for further developments of multi-partner learning and contributivity measurement approaches.

### Dataset construction
Now that we know all of that, we can create our dataset class.
#### Download and unzip data if needed

In [3]:
!curl https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip --output trainingandtestdata.zip
!unzip trainingandtestdata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 77.5M  100 77.5M    0     0  15.0M      0  0:00:05  0:00:05 --:--:-- 15.9M
Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


#### Define our Dataset class

In [1]:
class Sentiment140(Dataset):
    def __init__(self):

        x, y = self.load_data()
        self.max_tokens = self.getMax(x)

        self.num_words = None
        self.word_index = self.tokenize()
        self.num_words = len(self.word_index)

        x = self.create_sequences(x)
        y = self.preprocess_dataset_labels(y)

        self.input_shape = self.max_tokens
        self.num_classes = len(np.unique(y))


        print('length of the dictionary ',len(self.word_index))
        print('max token ', self.max_tokens)
        print('num classes', self.num_classes)

        (x_train, x_test) = train_test_split(x, shuffle = False)
        (y_train, y_test) = train_test_split(y, shuffle = False)

        super(Sentiment140, self).__init__(dataset_name='sentiment140',
                                           num_classes=self.num_classes,
                                           input_shape=self.input_shape,
                                           x_train=x_train,
                                           y_train=y_train,
                                           x_test=x_test,
                                           y_test=y_test)

    @staticmethod
    def load_data(): #  load the data, transform the .csv into usable dataframe

        df_train = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "raw_unicode_escape", header=None)
        df_test = pd.read_csv("testdata.manual.2009.06.14.csv", encoding = "raw_unicode_escape",  header=None)

        df_train.columns = ["polarity", "id", "date", "query", "user", "text"]
        df_test.columns = ["polarity", "id", "date", "query", "user", "text"]

        # We keep only a fraction of the whole dataset

        df_train = df_train.sample(frac = 0.1)

        x = df_train["text"]
        y = df_train["polarity"]

        return x, y

    #  Preprocessing methods
    @staticmethod
    def process( txt):
        out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
        out = out.split()
        out = [word.lower() for word in out]
        return out

    @staticmethod
    def getMax( data):
        max_tokens = 0
        for txt in data:
            if max_tokens < len(txt.split()):
                max_tokens = len(txt.split())
        return max_tokens


    def tokenize(self, thresh = 5):
        count  = dict()
        idx = 1
        word_index = dict()
        for txt in x:
            words = self.process(txt)
            for word in words:
                if word in count.keys():
                    count[word] += 1
                else:
                    count[word]  = 1
        most_counts = [word for word in count.keys() if count[word]>=thresh]
        for word in most_counts:
            word_index[word] = idx
            idx+=1
        return word_index


    def create_sequences(self,data):
        tokens = []
        for txt in data:
            words = self.process(txt)
            seq = [0] * self.max_tokens
            i = 0
            for word in words:
                start = self.max_tokens-len(words)
                if word.lower() in self.word_index.keys():
                    seq[i+start] = self.word_index[word]
                i+=1
            tokens.append(seq)
        return np.array(tokens)

    @staticmethod
    def preprocess_dataset_labels( label):
        label = np.array([e/4 for e in label])
        return label


    def generate_new_model(self): # Define the model generator
        model = Sequential()
        embedding_size = 8
        model.add(Embedding(input_dim=self.num_words,
                        output_dim=embedding_size,
                        input_length=self.max_tokens,
                        name='layer_embedding'))

        model.add(GRU(units=16, name = "gru_1",return_sequences=True))
        model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
        model.add(GRU(units=4, name= "gru_3"))
        model.add(Dense(1, activation='sigmoid',name="dense_1"))
        model.compile(loss='binary_crossentropy',
                  optimizer="Adam",
                  metrics=['accuracy'])
        return model


#### Create dataset

And we can eventually generate our object!

In [10]:
my_dataset = Sentiment140()

## 4 - Create the custom scenario
The dataset can be passed to the scenario, through the `dataset` argument.

In [ ]:
from mplc.scenario import Scenario

my_scenario = Scenario(3, [0.1, 0.4, 0.5], dataset=my_dataset)

# That's it!

Now you can explore our other tutorials for a better overview of what can be done with `mplc`!

This work is collaborative, enthusiasts are welcome to comment open issues and PRs or open new ones.

Should you be interested in this open effort and would like to share any question, suggestion or input, you can use the following channels:

- This Github repository (issues or PRs)
- Substra Foundation's [Slack workspace](https://substra-workspace.slack.com/join/shared_invite/zt-cpyedcab-FHYgpy08efKJ2FCadE2yCA), channel `#workgroup-mpl-contributivity`
- Email: hello@substra.org

![logo Substra Foundation](../../img/substra_logo_couleur_rvb_w150px.png)